<h4 style="text-align: center;">NOTEBOOK</h4>
<p style="text-align: center; font-style: italic">~~ TRAINING AI MODEL ~~</p>
</br>

<p style="text-align: justify; margin-left: 80px; margin-right: 80px;">
Ce notebook a pour objectif de regrouper les différents modèles d'architecture d'intelligence artificiel pour la détection des glyphes cunéiformes.</p>

</br>

<h3 style="text-align: justify; margin-left: 40px; margin-right: 80px;">
<b>Sommaire</b>
</h3>

<span style="margin-left: 80px;">- 1</span></br>
<span style="margin-left: 80px;">- 2</span></br>
<span style="margin-left: 80px;">- 3</span></br>

</br>

---
</br>

In [32]:
import base64
import dagshub
import mlflow
import os
import pandas as pd
import sys

from io import BytesIO
from PIL import Image
from tqdm import tqdm

# - Import Personal Library
sys.path.append("../utils/")    ## - db_data_fetch
import db_data_fetch as dbdf

## Get data from database (bbox and images)

In [25]:
SPLIT_SET = ['train', 'test']

df_train = pd.DataFrame()
df_test = pd.DataFrame()

train_images = pd.DataFrame()
test_images = pd.DataFrame()

#### Fetch Data Bbox Annotation

In [26]:
for param in SPLIT_SET:
    result = dbdf.fetch_data_bbox_annotation(param)

    if param == 'train':
        df_train = result
    elif param == 'test':
        df_test = result

#### Fetch Image

In [27]:
for param in SPLIT_SET:
    result = dbdf.fetch_image(param)

    if param == 'train':
        train_images = result
    elif param == 'test':
        test_images = result

2
2


#### Save dataframe in local

In [37]:
df_train.to_csv('data/train.csv', index=False)
df_test.to_csv('data/test.csv', index=False)

#### Decode encoded_image and saved it on local in two folder (train and test)

In [35]:
directory = 'data/train_images/'

for index, row in tqdm(train_images.iterrows(), total=train_images.shape[0]):
    picture_data = row['encoded_image']
    binary_data = base64.b64decode(picture_data)

    image = Image.open(BytesIO(binary_data))

    if not os.path.exists(directory):
        os.makedirs(directory)

    image.save(os.path.join(directory, f'{row["tablet_CDLI"]}.jpg'))

    # image.show()

100%|██████████| 46/46 [00:10<00:00,  4.57it/s]


In [36]:
directory = 'data/test_images/'

for index, row in tqdm(test_images.iterrows(), total=test_images.shape[0]):
    picture_data = row['encoded_image']
    binary_data = base64.b64decode(picture_data)

    image = Image.open(BytesIO(binary_data))

    if not os.path.exists(directory):
        os.makedirs(directory)

    image.save(os.path.join(directory, f'{row["tablet_CDLI"]}.jpg'))

    # image.show()

100%|██████████| 33/33 [00:05<00:00,  6.54it/s]


------------
## NAIVE AI

In [ ]:
dagshub.init("my-first-repo", "CharleyDL", mlflow=True)
mlflow.start_run()